In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [8]:
training_epochs = 50
batch_size = 500
learning_rate = 0.01
dropout=0.75
n_input_width=28
n_input_height=28

n_output  = 10 # e.g. MNIST total classes (0-9 digits)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input_width * n_input_height])
y = tf.placeholder(tf.float32, [None, n_output])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# Create some wrappers for simplicity
def conv2d(x, W, b):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, n_input_width, n_input_width, 1])
    

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1)
  
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [10]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1':tf.Variable(tf.random_normal([5,5,1,32], stddev=0.1)),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2':tf.Variable(tf.random_normal([5,5,32, 64], stddev=0.1)),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([1024, 10], stddev=0.1))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32], stddev=0.1)),
    'bc2':tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([10], stddev=0.1))

}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()


In [11]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    for epoch in range(training_epochs):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            # Calculate batch loss and accuracy
        loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                          y: batch_y,keep_prob: 1.
                                                          })
        print "Epoch  " ,epoch  ,", batch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc)
    print "Optimization Finished!"

    # Calculate accuracy mnist test images
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y: mnist.test.labels,
                                      keep_prob: 1.
                                      })

Epoch   0 , batch Loss= 75.406456, Training Accuracy= 0.12400
Epoch   1 , batch Loss= 26.540581, Training Accuracy= 0.11400
Epoch   2 , batch Loss= 6.437286, Training Accuracy= 0.27800
Epoch   3 , batch Loss= 2.900187, Training Accuracy= 0.17800
Epoch   4 , batch Loss= 2.400126, Training Accuracy= 0.13800
Epoch   5 , batch Loss= 2.167945, Training Accuracy= 0.34400
Epoch   6 , batch Loss= 1.966582, Training Accuracy= 0.49800
Epoch   7 , batch Loss= 1.743504, Training Accuracy= 0.56600
Epoch   8 , batch Loss= 1.426725, Training Accuracy= 0.60400
Epoch   9 , batch Loss= 1.023214, Training Accuracy= 0.66800
Epoch   10 , batch Loss= 0.909899, Training Accuracy= 0.79400
Epoch   11 , batch Loss= 0.787288, Training Accuracy= 0.81800
Epoch   12 , batch Loss= 0.651040, Training Accuracy= 0.82400
Epoch   13 , batch Loss= 0.675232, Training Accuracy= 0.81600
Epoch   14 , batch Loss= 0.472915, Training Accuracy= 0.87800
Epoch   15 , batch Loss= 0.589384, Training Accuracy= 0.83200
Epoch   16 , bat